In [ ]:
!pip install ../input/timm-0-4-12/timm-0.4.12-py3-none-any.whl ../input/python-box-5-4-1/python_box-5.4.1-py3-none-any.whl ../input/webcolors/webcolors-1.11.1-py3-none-any.whl 

In [ ]:
import sys
sys.path.append('../input/yetanotherefficientdetpytorch/')

import os
import time
import torch
from torch.backends import cudnn
from matplotlib import colors

from backbone import EfficientDetBackbone
import cv2
import numpy as np
import pandas as pd

from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, invert_affine, postprocess, STANDARD_COLORS, standard_to_bgr, get_index_label, plot_one_box

In [ ]:
test_path = '../input/petfinder-pawpularity-score/test/'
train_path = '../input/petfinder-pawpularity-score/train/'
train_csv_path = '../input/petfinder-pawpularity-score/train.csv'
test_csv_path = '../input/petfinder-pawpularity-score/test.csv'

test_metadata = pd.read_csv(test_csv_path)
train_metadata = pd.read_csv(train_csv_path)

In [ ]:
def display(preds, imgs, imshow=True, imwrite=False):
    max_area = 0
    x_1, y_1, x_2, y_2 = 0, 0, 0, 0
    for i in range(len(imgs)):
        
        if len(preds[i]['rois']) == 0:
            continue

        imgs[i] = imgs[i].copy()

        for j in range(len(preds[i]['rois'])):
            x1, y1, x2, y2 = preds[i]['rois'][j].astype(np.int)
            obj = obj_list[preds[i]['class_ids'][j]]
            score = float(preds[i]['scores'][j])
#             print(obj)
#             plot_one_box(imgs[i], [x1, y1, x2, y2], label=obj,score=score,color=color_list[get_index_label(obj, obj_list)])
            
            if obj == 'cat' or obj == 'dog':
                area = (x2 - x1) * (y2 - y1)
                if area >= max_area:
                    max_area = area
                    x_1, y_1, x_2, y_2 = x1, y1, x2, y2
                
    
#         if imshow:
#             cv2.imshow('img', imgs[i])
#             cv2.waitKey(0)

#         if imwrite:
#             cv2.imwrite(f'./img_inferred_d{compound_coef}_this_repo_{i}.jpg', imgs[i])
        
    return x_1, y_1, x_2, y_2

In [ ]:
def crop(img_Id):
    compound_coef = 6
    force_input_size = None  # set None to use default size

    # replace this part with your project's anchor config
    anchor_ratios = [(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]
    anchor_scales = [2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]

    threshold = 0.2
    iou_threshold = 0.2

    use_cuda = True
    use_float16 = False
    cudnn.fastest = True
    cudnn.benchmark = True

    obj_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
                'fire hydrant', '', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
                'cow', 'elephant', 'bear', 'zebra', 'giraffe', '', 'backpack', 'umbrella', '', '', 'handbag', 'tie',
                'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
                'skateboard', 'surfboard', 'tennis racket', 'bottle', '', 'wine glass', 'cup', 'fork', 'knife', 'spoon',
                'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut',
                'cake', 'chair', 'couch', 'potted plant', 'bed', '', 'dining table', '', '', 'toilet', '', 'tv',
                'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink',
                'refrigerator', '', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
                'toothbrush']

    color_list = standard_to_bgr(STANDARD_COLORS)
    # tf bilinear interpolation is different from any other's, just make do
    input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]
    input_size = input_sizes[compound_coef] if force_input_size is None else force_input_size

    model_det = EfficientDetBackbone(compound_coef=compound_coef, num_classes=len(obj_list),
                                 ratios=anchor_ratios, scales=anchor_scales)
    model_det.load_state_dict(torch.load(f'../input/efficientdet-d6/efficientdet-d{compound_coef}.pth', map_location='cpu'))
    model_det.requires_grad_(False)
    model_det.eval()

    if use_cuda:
        model_det = model_det.cuda()
    if use_float16:
        model_det = model_det.half()
    
    img_path = test_path + img_Id + '.jpg' 
#     print(img_path)
    ori_imgs, framed_imgs, framed_metas = preprocess(img_path, max_size=input_size)

    if use_cuda:
        x = torch.stack([torch.from_numpy(fi).cuda() for fi in framed_imgs], 0)
    else:
        x = torch.stack([torch.from_numpy(fi) for fi in framed_imgs], 0)

    x = x.to(torch.float32 if not use_float16 else torch.float16).permute(0, 3, 1, 2)
    
    with torch.no_grad():
        features, regression, classification, anchors = model_det(x)

        regressBoxes = BBoxTransform()
        clipBoxes = ClipBoxes()

        out = postprocess(x,
                          anchors, regression, classification,
                          regressBoxes, clipBoxes,
                          threshold, iou_threshold)
    
    out = invert_affine(framed_metas, out)
    x_1, y_1, x_2, y_2 = display(out, ori_imgs, imshow=False, imwrite=True)
    if x_1 == 0 and x_2 == 0 and y_1==0 and y_2==0:
        img = ori_imgs[0]
    else:
        img = ori_imgs[0][y_1:y_2, x_1:x_2]
    
    return img

In [ ]:
import os
import cv2
import timm
import pandas as pd
import numpy as np
from box import Box
from timm import create_model
from sklearn.model_selection import KFold

from matplotlib import pyplot as plt

import albumentations
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.io import read_image

from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
train_path = '../input/petfinder-pawpularity-score/train/'
test_path = '../input/petfinder-pawpularity-score/test/'
train_metadata_path = '../input/petfinder-pawpularity-score/train.csv'
test_metadata_path = '../input/petfinder-pawpularity-score/test.csv'

train_num = len(os.listdir(train_path))
test_num = len(os.listdir(test_path))
train_metadata = pd.read_csv(train_metadata_path)
test_metadata = pd.read_csv(test_metadata_path)

print(f'Train Data num is : {train_num} \nTest Data num is : {test_num}')

In [ ]:
CONFIG = {
    'RANDOM_SEED' : 2021,
    'N_SPLITS' : 5,
    'img_size' : 384,
    'epochs' : 20,
    'train_dataloader' : {
        'batch_size' : 32,
        'shuffle' : True,
        'num_workers' : 2,        
    },
    'val_dataloader' : {
        'batch_size' : 32,
        'shuffle' : False,
        'num_workers' : 2,
    },
    'test_dataloader' : {
        'batch_size' : 1,
        'shuffle' : False,
        'num_workers' : 2,
    },
    'model' : {
        'name' : 'efficientnet_b2',
        'output_dim' : 1,
    },
    'optimizer' : {
        'name' : 'optim.Adam',
        'params' : {
            'lr' : 1e-5,
        }
    },
    # 学习率衰减
    'scheduler' : {
        'name' : 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
        'params' : {
            'T_0' : 20,
            'eta_min' : 1e-6,
        }
    },
    'bce_loss' : 'nn.BCEWithLogitsLoss',
    'rmse_loss' : 'nn.MSELoss',
    
}
# 使用Box为字典增加点调用属性
config = Box(CONFIG)

In [ ]:
def get_default_transforms(mode):
    if mode == 'train':
        aug = albumentations.Compose([
            albumentations.Resize(config.img_size, config.img_size, p=1),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.RandomRotate90(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0,
            ),
            ToTensorV2(),
        ])
    else:
        aug = albumentations.Compose([
            albumentations.Resize(config.img_size, config.img_size, p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0,
            ),
            ToTensorV2(),
        ])
        
    return aug

In [ ]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB

class PetDataSet(Dataset):
    def __init__(self, df, mode, data_path, transforms=None):
        # df的某一列为series,series的values为numpy.array
        self.file_name = df['Id'].values
        self.mode = mode
        if self.mode != 'test':
            self.Pawpularity = df['Pawpularity'].values
        self.transforms = transforms
        self.data_path = data_path
    
    def __len__(self):
        return len(self.file_name)
    
    def __getitem__(self, idx):
        img_name = self.file_name[idx]
        img = cv2.imread(os.path.join(self.data_path, img_name + '.jpg'))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transforms != None:
            img = self.transforms(image=img)['image']
        
        img_Id = img_name.split('/')[-1].split('.')[0]
        if self.mode != 'test':
            label = self.Pawpularity[idx] / 100.
            
            return img, label
        else:
            img_Id = img_name.split('/')[-1].split('.')[0]
            return img, img_Id

# 声明一个DataModule,为了减少重复声明DataSet和Dataloader的冗余
class PetDataModule(pl.LightningDataModule):
    def __init__(self, train_df, val_df, test_df, cfg):
        super().__init__()
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        self.cfg = cfg
    
    def train_dataloader(self):
        train_transforms = get_default_transforms(mode='test')
        dataset = PetDataSet(self.train_df, mode='train', data_path=train_path, transforms=train_transforms)
        
        return DataLoader(dataset, **self.cfg.train_dataloader)
    
    def val_dataloader(self):
        val_transforms = get_default_transforms(mode='test')
        dataset = PetDataSet(self.val_df, mode='val', data_path=train_path, transforms=val_transforms)
        
        return DataLoader(dataset, **self.cfg.val_dataloader)
    
    def test_dataloader(self):
        test_transforms = get_default_transforms(mode='test')
        dataset = PetDataSet(self.test_df, mode='test', data_path=test_path, transforms=test_transforms)
        
        return DataLoader(dataset, **self.cfg.test_dataloader)

In [ ]:
# 为numpy,random,pytorch设置随机种子
seed_everything(config.RANDOM_SEED)

# mixup混合图像
# 一种图像增强的trick，利用beta分布获取一个值，然后利用该值将两个图像叠加，增强泛化性
def mix_up(x, y, alpha):
    lam = np.random.beta(alpha, alpha)
    rand_idx = torch.randperm(x.size()[0]) # 返回一个随机打乱的数组
    mix_x = lam * x + (1 - lam) * x[rand_idx, :]
    target_a, target_b = y, y[rand_idx]
    
    return mix_x, target_a, target_b, lam

# 使用pytorch_lightning构建模型(下面是模板)
# 声明模型必须包含的三个组件:1、__init__初始化 2、training_step(),每个batch的训练 3、configure_optimizers() 优化器的定义
class Model(pl.LightningModule):
    def __init__(self, cfg):
        super(Model, self).__init__()
        self.cfg = cfg
        self.build_model()
        # python的eval会执行一个字符串表达式,在config中定义的损失是以字符串的形式
        self.criterion_bce = eval(self.cfg.bce_loss)()
        self.criterion_rmse = eval(self.cfg.rmse_loss)()
        
    def build_model(self):
        # timm中的网络结构，在一层层的加深之后，最后是一个全局的avg_pool层，返回一个x维度的向量，然后pool层后面是classifier层
        # num_classes=0就是说分类层中的输出类别为1，只返回pool的特征向量
        self.backbone = create_model(self.cfg.model.name, pretrained=False, num_classes=0, in_chans=3)
        self.num_features = self.backbone.num_features
        self.fc = nn.Sequential(nn.Dropout(0.5),
                                nn.Linear(self.num_features, self.cfg.model.output_dim))
    
    def forward(self, x):
        x = self.backbone(x)
        out = self.fc(x)
        
        return out, x
    
    # 必须返回一个loss值(往往会以字典的形式返回多个数据)，batch为(img, label)的集合
    def training_step(self, batch, batch_idx):
        imgs, labels = batch
        
        # 以一半的概率进行mixup
        if torch.rand(1)[0] > 0.5:
            mix_imgs, target_a, target_b, lam = mix_up(imgs, labels, alpha=0.5)
            pawpularity_preds = self.forward(mix_imgs)
            loss_bce = self.criterion_bce(pawpularity_preds.view(-1), target_a.float()) * lam + \
                        self.criterion_bce(pawpularity_preds.view(-1), target_b.float()) * (1 - lam)
        else:  
            pawpularity_preds = self.forward(imgs)
            loss_bce = self.criterion_bce(pawpularity_preds.view(-1), labels.float())
        
        preds = pawpularity_preds.sigmoid().detach().cpu() * 100.
        labels = labels.detach().cpu() * 100.
        
        return {'loss' : loss_bce, 'preds' : preds, 'labels' : labels}
    
    # 每个batch返回的loss会放在一个列表中
    # training_epoch_end不返回值，或者说期望的返回值为None，所以应该使用log来记录损失
    def training_epoch_end(self, training_step_outputs):
        preds = []
        labels = []
        for out in training_step_outputs:
            pred, label =  out['preds'], out['labels']
            preds.append(pred)
            labels.append(label)
        
        preds = torch.cat(preds)
        labels = torch.cat(labels)
        train_Loss = torch.sqrt(self.criterion_rmse(preds.view(-1), labels))
        
        print(f'Epoch [{self.current_epoch + 1}] train_Loss : {train_Loss}')
        self.log('train_Loss', train_Loss)
    
    def validation_step(self, batch, batch_idx):
        imgs, labels = batch
        pawpularity_preds = self.forward(imgs)
        preds = pawpularity_preds.sigmoid().detach().cpu() * 100.
        labels = labels.detach().cpu() * 100.
        loss_rmse = torch.sqrt(self.criterion_rmse(preds.view(-1), labels))
        
        return {'loss' : loss_rmse, 'preds' : preds, 'labels' : labels}
    
    def validation_epoch_end(self, validation_step_outputs):
        preds = []
        labels = []
        for out in validation_step_outputs:
            pred, label =  out['preds'], out['labels']
            preds.append(pred)
            labels.append(label)
        
        preds = torch.cat(preds)
        labels = torch.cat(labels)
        val_Loss = torch.sqrt(self.criterion_rmse(preds.view(-1), labels))
        
        print(f'Epoch [{self.current_epoch + 1}] val_Loss : {val_Loss}')
        self.log('val_Loss', val_Loss)
    
    # 声明优化器，当training_step返回loss的时候，会自动进行反向传播，可以在声明Trainer的时候关闭自动反向传播
    def configure_optimizers(self):
        # *表示将序列解包成位置参数, **表示将字典解包成关键字参数
        # 位置参数按照函数定义的位置来传递参数,关键字参数则是以键-值的方式传递参数
        # 简单的来理解当：mode=‘train’,这个样子传入的时候，mode就变成了一个关键字参数
        optimizer = eval(self.cfg.optimizer.name)(self.parameters(), **self.cfg.optimizer.params)
        scheduler = eval(self.cfg.scheduler.name)(optimizer, **self.cfg.scheduler.params)
        
        return [optimizer], [scheduler]

In [ ]:
test_transforms = get_default_transforms(mode='test')
preds = np.zeros(test_num)
for j in range(5):
    pred = []
    ckpt_path = '../input/eff-al/' + 'best_epoch_' + str(j) + '.pth'

    model = Model(config)
    checkpoint = torch.load(ckpt_path)
    r = model.load_state_dict(checkpoint)
    print(r)
    model.eval()
    model.cuda()
    with torch.no_grad():
        for i in range(len(test_metadata)):
            img_Id = test_metadata.loc[i, 'Id']
            print(img_Id)
            img = crop(img_Id)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = test_transforms(image=img)['image'].unsqueeze(axis=0)

            img = img.cuda()
            pred_pawpularity, embedding = model(img.float())
            pred_pawpularity = pred_pawpularity.view(-1)
            pred.extend(pred_pawpularity.sigmoid().data.cpu().numpy() * 100.)

        preds += np.array(pred) / 5

In [ ]:
sample_sub_path = '../input/petfinder-pawpularity-score/sample_submission.csv'
submission = pd.read_csv(sample_sub_path)
submission['Pawpularity'] = preds
submission.to_csv('submission.csv', index=False)

In [ ]:
submission